In [ ]:
!sudo apt-get update -qq
!sudo apt-get install -qq -y fonts-nanum

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                10000 non-null  object 
 1   주거 형태              10000 non-null  object 
 2   연간 소득              10000 non-null  float64
 3   현재 직장 근속 연수        10000 non-null  object 
 4   체납 세금 압류 횟수        10000 non-null  float64
 5   개설된 신용계좌 수         10000 non-null  int64  
 6   신용 거래 연수           10000 non-null  float64
 7   최대 신용한도            10000 non-null  float64
 8   신용 문제 발생 횟수        10000 non-null  int64  
 9   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 10  개인 파산 횟수           10000 non-null  int64  
 11  대출 목적              10000 non-null  object 
 12  대출 상환 기간           10000 non-null  object 
 13  현재 대출 잔액           10000 non-null  float64
 14  현재 미상환 신용액         10000 non-null  float64
 15  월 상환 부채액           10000 non-null  float64
 16  신용 점수              1000

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                2062 non-null   object 
 1   주거 형태              2062 non-null   object 
 2   연간 소득              2062 non-null   float64
 3   현재 직장 근속 연수        2062 non-null   object 
 4   체납 세금 압류 횟수        2062 non-null   float64
 5   개설된 신용계좌 수         2062 non-null   int64  
 6   신용 거래 연수           2062 non-null   float64
 7   최대 신용한도            2062 non-null   float64
 8   신용 문제 발생 횟수        2062 non-null   int64  
 9   마지막 연체 이후 경과 개월 수  2062 non-null   int64  
 10  개인 파산 횟수           2062 non-null   int64  
 11  대출 목적              2062 non-null   object 
 12  대출 상환 기간           2062 non-null   object 
 13  현재 대출 잔액           2062 non-null   float64
 14  현재 미상환 신용액         2062 non-null   float64
 15  월 상환 부채액           2062 non-null   float64
 16  신용 점수              2062 

In [ ]:
y=train.pop('채무 불이행 여부')
test_UID=test.pop('UID')

In [ ]:
train.head()

,UID,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
0,TRAIN_00000,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767
1,TRAIN_00001,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767
2,TRAIN_00002,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800
3,TRAIN_00003,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796
4,TRAIN_00004,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751


In [ ]:
test.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
0,월세,1560090.0,10년 이상,0.0,13,12.0,495561.0,0,18,0,기타,단기 상환,376332.0,133522.5,29641.5,736
1,주택 담보 대출 (거주 중),2102616.0,2년,0.0,9,29.0,580833.0,0,40,0,부채 통합,장기 상환,830379.0,302983.5,20151.0,718
2,주택 담보 대출 (거주 중),2477989.5,10년 이상,0.0,11,26.5,995841.0,0,44,0,부채 통합,장기 상환,877635.0,379278.0,13113.0,722
3,주택 담보 대출 (거주 중),1571091.0,6년,0.0,7,34.4,601656.0,0,45,0,부채 통합,단기 상환,487278.0,275395.5,11679.0,762
4,주택 담보 대출 (거주 중),2290260.0,10년 이상,0.0,19,25.0,1954623.0,0,14,0,부채 통합,단기 상환,397782.0,742767.0,42370.5,775


In [ ]:
train.nunique()

,0
UID,10000
주거 형태,4
연간 소득,9970
현재 직장 근속 연수,11
체납 세금 압류 횟수,8
개설된 신용계좌 수,33
신용 거래 연수,397
최대 신용한도,9585
신용 문제 발생 횟수,8
마지막 연체 이후 경과 개월 수,88


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

columns = ['주거 형태', '현재 직장 근속 연수', '대출 목적', '대출 상환 기간']

for col in columns:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                10000 non-null  object 
 1   주거 형태              10000 non-null  int64  
 2   연간 소득              10000 non-null  float64
 3   현재 직장 근속 연수        10000 non-null  int64  
 4   체납 세금 압류 횟수        10000 non-null  float64
 5   개설된 신용계좌 수         10000 non-null  int64  
 6   신용 거래 연수           10000 non-null  float64
 7   최대 신용한도            10000 non-null  float64
 8   신용 문제 발생 횟수        10000 non-null  int64  
 9   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 10  개인 파산 횟수           10000 non-null  int64  
 11  대출 목적              10000 non-null  int64  
 12  대출 상환 기간           10000 non-null  int64  
 13  현재 대출 잔액           10000 non-null  float64
 14  현재 미상환 신용액         10000 non-null  float64
 15  월 상환 부채액           10000 non-null  float64
 16  신용 점수              1000

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2062 entries, 0 to 2061
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   주거 형태              2062 non-null   int64  
 1   연간 소득              2062 non-null   float64
 2   현재 직장 근속 연수        2062 non-null   int64  
 3   체납 세금 압류 횟수        2062 non-null   float64
 4   개설된 신용계좌 수         2062 non-null   int64  
 5   신용 거래 연수           2062 non-null   float64
 6   최대 신용한도            2062 non-null   float64
 7   신용 문제 발생 횟수        2062 non-null   int64  
 8   마지막 연체 이후 경과 개월 수  2062 non-null   int64  
 9   개인 파산 횟수           2062 non-null   int64  
 10  대출 목적              2062 non-null   int64  
 11  대출 상환 기간           2062 non-null   int64  
 12  현재 대출 잔액           2062 non-null   float64
 13  현재 미상환 신용액         2062 non-null   float64
 14  월 상환 부채액           2062 non-null   float64
 15  신용 점수              2062 non-null   int64  
dtypes: float64(7), int64(9)


In [ ]:
train=train.drop(columns='UID')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

x_tr,x_val,y_tr,y_val=train_test_split(train,y,test_size=0.2,random_state=42,stratify=y)
rf=RandomForestClassifier()
rf.fit(x_tr,y_tr)

RandomForestClassifier()

In [ ]:
from sklearn.metrics import roc_auc_score

pred=rf.predict(x_val)
print(roc_auc_score(y_val,pred))

0.6482762917243313


In [ ]:
pred2=rf.predict(test)
result=pd.DataFrame({'UID':test_UID,'채무 불이행 확률':pred2})

In [ ]:
result.to_csv('result.csv',index=False)
pd.read_csv('result.csv')

,UID,채무 불이행 확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
2057,TEST_2057,0
2058,TEST_2058,0
2059,TEST_2059,0
2060,TEST_2060,0


In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

x_tr, x_val, y_tr, y_val = train_test_split(train, y, test_size=0.2, random_state=42, stratify=y)

lgbm = LGBMClassifier(n_estimators=500, max_depth=6, learning_rate=0.05, random_state=42, verbose=-1)
lgbm.fit(x_tr, y_tr)

y_pred = lgbm.predict_proba(x_val)[:, 1]
auc = roc_auc_score(y_val, y_pred)
print("LightGBM AUC:", auc)

LightGBM AUC: 0.7333013674856153


In [ ]:
pred2=lgbm.predict(test)
result2=pd.DataFrame({'UID':test_UID,'채무 불이행 확률':pred2})

result2.to_csv('result2.csv',index=False)
pd.read_csv('result2.csv')

,UID,채무 불이행 확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
2057,TEST_2057,0
2058,TEST_2058,0
2059,TEST_2059,0
2060,TEST_2060,0


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=6, eval_metric='auc', use_label_encoder=False)
xgb.fit(x_tr, y_tr)

y_pred = xgb.predict_proba(x_val)[:, 1]
auc = roc_auc_score(y_val, y_pred)
print("XGBoost AUC:", auc)

XGBoost AUC: 0.7240008633003885


In [ ]:
pred2=xgb.predict(test)
result3=pd.DataFrame({'UID':test_UID,'채무 불이행 확률':pred2})

result3.to_csv('result3.csv',index=False)
pd.read_csv('result3.csv')

,UID,채무 불이행 확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
2057,TEST_2057,0
2058,TEST_2058,0
2059,TEST_2059,0
2060,TEST_2060,0


In [ ]:
train.describe()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
count,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.00000,10000.000000
mean,0.973500,2.163959e+06,3.210300,0.199300,12.248900,19.879360,1.175265e+06,0.62620,30.889200,0.373200,4.743600,0.302500,5.061200e+05,3.649126e+05,22367.28075,744.215000
std,0.893352,1.434430e+06,3.326797,0.714304,4.620572,7.206693,1.604199e+06,1.23419,20.011561,0.843797,2.257159,0.459364,2.831462e+05,3.537942e+05,15186.49738,56.995698
min,0.000000,2.676210e+05,0.000000,0.000000,2.000000,6.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000,2.917650e+04,0.000000e+00,0.00000,502.000000
25%,0.000000,1.311437e+06,0.000000,0.000000,9.000000,14.600000,4.482476e+05,0.00000,14.000000,0.000000,4.000000,0.000000,3.001901e+05,1.471856e+05,10893.75000,704.000000
50%,1.000000,1.743223e+06,3.000000,0.000000,12.000000,17.950000,7.670910e+05,0.00000,28.000000,0.000000,4.000000,0.000000,4.743412e+05,2.547930e+05,20160.00000,756.000000
75%,2.000000,2.447664e+06,6.000000,0.000000,15.000000,24.100000,1.147283e+06,1.00000,41.000000,0.000000,4.000000,1.000000,5.919079e+05,4.749180e+05,30647.25000,793.000000
max,3.000000,1.722975e+07,10.000000,7.000000,35.000000,51.700000,2.323233e+07,7.00000,88.000000,3.000000,13.000000,1.000000,1.373613e+06,3.946300e+06,153574.50000,825.000000


In [ ]:
test.describe()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
count,2062.000000,2.062000e+03,2062.000000,2062.000000,2062.000000,2062.000000,2.062000e+03,2062.000000,2062.000000,2062.000000,2062.000000,2062.00000,2.062000e+03,2.062000e+03,2062.000000,2062.000000
mean,1.112512,2.172612e+06,3.144035,0.042677,11.404947,19.133269,1.135118e+06,0.192532,34.625121,0.115907,4.512609,0.28516,4.685609e+05,3.808292e+05,28136.472599,756.448594
std,0.954038,1.294999e+06,3.200575,0.351226,4.713266,6.745304,1.257305e+07,0.569551,21.769995,0.347793,1.973769,0.45160,2.685751e+05,4.190525e+05,16932.244087,51.595834
min,0.000000,2.468955e+05,0.000000,0.000000,2.000000,4.800000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00000,3.257100e+04,0.000000e+00,0.000000,524.000000
25%,0.000000,1.389211e+06,0.000000,0.000000,8.000000,14.500000,3.758700e+05,0.000000,16.000000,0.000000,4.000000,0.00000,2.657902e+05,1.526460e+05,16408.125000,731.000000
50%,2.000000,1.880202e+06,2.000000,0.000000,11.000000,17.800000,6.374115e+05,0.000000,32.000000,0.000000,4.000000,0.00000,4.048275e+05,2.769915e+05,25499.250000,769.000000
75%,2.000000,2.556386e+06,6.000000,0.000000,14.000000,22.775000,1.047568e+06,0.000000,50.000000,0.000000,4.000000,1.00000,6.368092e+05,4.667730e+05,35977.500000,796.000000
max,3.000000,1.522402e+07,10.000000,7.000000,37.000000,50.600000,5.700784e+08,7.000000,118.000000,4.000000,13.000000,1.00000,1.183545e+06,6.550868e+06,140460.000000,820.000000


In [ ]:
train.head()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
0,1,1941337.5,0,0.0,9,13.4,400597.5,0,24,1,4,0,390903.0,225457.5,8806.5,767
1,0,1979505.0,0,0.0,5,15.1,360679.5,0,11,0,4,0,1002184.5,64749.0,24961.5,767
2,0,1356381.0,5,0.0,12,18.8,491770.5,1,74,3,4,0,227775.0,487644.0,12069.0,800
3,0,1049017.5,7,0.0,15,14.8,411546.0,1,22,1,4,0,251383.5,413211.0,31749.0,796
4,0,4320217.5,3,0.0,11,26.1,895288.5,0,32,0,4,1,1163176.5,78991.5,5862.0,751


In [ ]:
import numpy as np

# 로그 변환 적용
train["최대 신용한도"] = np.log1p(train["최대 신용한도"])
test["최대 신용한도"] = np.log1p(test["최대 신용한도"])

train["현재 미상환 신용액"] = np.log1p(train["현재 미상환 신용액"])
test["현재 미상환 신용액"] = np.log1p(test["현재 미상환 신용액"])

In [ ]:
train.describe()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
count,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.00000,10000.000000
mean,0.973500,2.163959e+06,3.210300,0.199300,12.248900,19.879360,13.017663,0.62620,30.889200,0.373200,4.743600,0.302500,5.061200e+05,12.297615,22367.28075,744.215000
std,0.893352,1.434430e+06,3.326797,0.714304,4.620572,7.206693,2.788388,1.23419,20.011561,0.843797,2.257159,0.459364,2.831462e+05,1.667779,15186.49738,56.995698
min,0.000000,2.676210e+05,0.000000,0.000000,2.000000,6.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.917650e+04,0.000000,0.00000,502.000000
25%,0.000000,1.311437e+06,0.000000,0.000000,9.000000,14.600000,13.013103,0.00000,14.000000,0.000000,4.000000,0.000000,3.001901e+05,11.899457,10893.75000,704.000000
50%,1.000000,1.743223e+06,3.000000,0.000000,12.000000,17.950000,13.550362,0.00000,28.000000,0.000000,4.000000,0.000000,4.743412e+05,12.448211,20160.00000,756.000000
75%,2.000000,2.447664e+06,6.000000,0.000000,15.000000,24.100000,13.952908,1.00000,41.000000,0.000000,4.000000,1.000000,5.919079e+05,13.070900,30647.25000,793.000000
max,3.000000,1.722975e+07,10.000000,7.000000,35.000000,51.700000,16.961056,7.00000,88.000000,3.000000,13.000000,1.000000,1.373613e+06,15.188289,153574.50000,825.000000


In [ ]:
test.describe()

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수
count,2062.000000,2.062000e+03,2062.000000,2062.000000,2062.000000,2062.000000,2062.000000,2062.000000,2062.000000,2062.000000,2062.000000,2062.00000,2.062000e+03,2062.000000,2062.000000,2062.000000
mean,1.112512,2.172612e+06,3.144035,0.042677,11.404947,19.133269,13.235749,0.192532,34.625121,0.115907,4.512609,0.28516,4.685609e+05,12.358685,28136.472599,756.448594
std,0.954038,1.294999e+06,3.200575,0.351226,4.713266,6.745304,1.484718,0.569551,21.769995,0.347793,1.973769,0.45160,2.685751e+05,1.451297,16932.244087,51.595834
min,0.000000,2.468955e+05,0.000000,0.000000,2.000000,4.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,3.257100e+04,0.000000,0.000000,524.000000
25%,0.000000,1.389211e+06,0.000000,0.000000,8.000000,14.500000,12.837001,0.000000,16.000000,0.000000,4.000000,0.00000,2.657902e+05,11.935883,16408.125000,731.000000
50%,2.000000,1.880202e+06,2.000000,0.000000,11.000000,17.800000,13.365172,0.000000,32.000000,0.000000,4.000000,0.00000,4.048275e+05,12.531746,25499.250000,769.000000
75%,2.000000,2.556386e+06,6.000000,0.000000,14.000000,22.775000,13.861983,0.000000,50.000000,0.000000,4.000000,1.00000,6.368092e+05,13.053600,35977.500000,796.000000
max,3.000000,1.522402e+07,10.000000,7.000000,37.000000,50.600000,20.161285,7.000000,118.000000,4.000000,13.000000,1.00000,1.183545e+06,15.695108,140460.000000,820.000000


In [ ]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

x_tr, x_val, y_tr, y_val = train_test_split(train, y, test_size=0.2, random_state=42, stratify=y)

lgbm = LGBMClassifier(n_estimators=500, max_depth=6, learning_rate=0.05, random_state=42, verbose=-1)
lgbm.fit(x_tr, y_tr)

y_pred = lgbm.predict_proba(x_val)[:, 1]
auc = roc_auc_score(y_val, y_pred)
print("LightGBM AUC:", auc)

LightGBM AUC: 0.7333013674856153


In [ ]:
pred2=lgbm.predict(test)
result4=pd.DataFrame({'UID':test_UID,'채무 불이행 확률':pred2})

result4.to_csv('result4.csv',index=False)
pd.read_csv('result4.csv')

,UID,채무 불이행 확률
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
2057,TEST_2057,0
2058,TEST_2058,0
2059,TEST_2059,0
2060,TEST_2060,0
